In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                             ],
                                             value='ALL',
                                             placeholder="Select Launch Site",
                                             searchable=True
                                             ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'},
                                                value=[2500, 8000]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches by Site')
        return fig
    else:
        fig = px.pie(filtered_df, names='class', title='Total Success Launches for Site {}' .format(entered_site))
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])
def get_scatter(entered_site, slider_range):
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    low, high = slider_range
    msk = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
    if entered_site == 'ALL':
        fig = px.scatter(spacex_df[msk], x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        fig = px.scatter(filtered_df[msk], x='Payload Mass (kg)', y='class', color='Booster Version Category', title='Correlation between Payload and Success for Site {}' .format(entered_site))
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [06/Oct/2022 14:25:49] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [06/Oct/2022 14:25:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Oct/2022 14:25:50] "POST /_dash-update-component HTTP/1.1" 200 -


In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.groupby('Launch Site').mean()

,Unnamed: 0,Flight Number,class,Payload Mass (kg)
Launch Site,,,,
CCAFS LC-40,12.5,14.576923,0.269231,2590.884615
CCAFS SLC-40,52.0,50.142857,0.428571,3516.664286
KSC LC-39A,42.0,37.692308,0.769231,4376.511538
VAFB SLC-4E,30.5,37.600000,0.400000,5813.800000


In [7]:
class_sum = spacex_df.groupby('Launch Site')[['class']].sum()

In [12]:
class_sum['class']/class_sum['class'].sum()

Launch Site
CCAFS LC-40     0.291667
CCAFS SLC-40    0.125000
KSC LC-39A      0.416667
VAFB SLC-4E     0.166667
Name: class, dtype: float64

In [11]:
12/24

0.5